In [1]:
import pandas as pd
import numpy as np
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print(device)

from transformers import BertTokenizer, BertModel

1.3.0
cuda


To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [2]:
# 讀取tsv # \t tab做區隔
df_news = pd.read_csv("data/2015_Company.tsv", sep="\t", encoding="utf-8")
df_news

,id,label,text
0,0,財務營運風險,財政部充分尊重銀行團確保債權的必要作為，預計元旦後就會請法院實施假扣押，如三重土地順利拍賣，...
1,1,財務營運風險,至三趴以上，「在假扣押前，仍歡迎頂新還錢。」頂新：將持續協商財政部次長吳當傑尊重並同意銀行團...
2,2,法律風險,測報告中指出，各款手機在裝上SIM卡前後開機連線時，傳送之資料皆未涉及第一類的敏感資訊，僅為...
3,3,法律風險,大哥大，透過4G漫遊方式共網爭議，戰火延燒，威寶電信（台灣之星）及中華電信、遠傳電信等業者，...
4,4,法律風險,測報告中指出，各款手機在裝上SIM卡前後開機連線時，傳送之資料皆未涉及第一類的敏感資訊，僅為...
...,...,...,...
15887,15887,資安風險,"感覺華碩客服有點推託,所謂的個資法是企業有義務妥善保管蒐集來的個人資料不\n感覺華碩客服有點..."
15888,15888,勞動風險,上市公司玉晶光（3406）今天傳出裁員，對此，公司發言人趙志強表示，並非裁員而\n上市公司玉...
15889,15889,勞動風險,繼台企銀（2834）工會因合併問題而罷工，創金融史首例後，大眾銀行（2847）工\n繼台企銀...
15890,15890,勞動風險,6億元賣給英業達，獲利21億元，預計明年首季入帳，活化資產。外界認為這將有機會帶動宏達電明年...


In [3]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [4]:
# # Tokenize input
# text = "很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。"
# tokenized_text = tokenizer.tokenize(text)      # 每個字切詞成一個list
# print(type(tokenized_text))
# tokenized_text

In [5]:
# indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)   # 每個字轉成id
# print(type(indexed_tokens))                         # list
# indexed_tokens

In [6]:
# indexed_tokens2 = tokenizer.build_inputs_with_special_tokens(indexed_tokens)    # 句子前後加上 CLS SEP
# print(type(indexed_tokens2))
# indexed_tokens2

In [7]:
# segments_ids = tokenizer.create_token_type_ids_from_sequences(indexed_tokens2)
# print(type(segments_ids))                                # list
# segments_ids

In [8]:
def tokenize_all(text):
#    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    tokenized_text = tokenizer.tokenize(text)    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    indexed_tokens2 = tokenizer.build_inputs_with_special_tokens(indexed_tokens)
    indexed_tokens2 = np.array(indexed_tokens2)          # list 轉 numpy
    return indexed_tokens2

In [9]:
def segments_all(indexed_tokens):
    segments_ids = tokenizer.create_token_type_ids_from_sequences(indexed_tokens)
    segments_ids = np.array(segments_ids)              # list 轉numpy
    return segments_ids

In [10]:
# df['text'] = df.comment.apply(lambda x: " ".join(jieba.cut(x)))
# df_news['tokenized_text'] = df_news['text'].apply(lambda x:  np.array( tokenizer.tokenize(x) ) )
df_news['indexed_tokens'] = df_news['text'].apply(tokenize_all)
df_news['segments_ids'] = df_news['indexed_tokens'].apply(segments_all)
df_news

,id,label,text,indexed_tokens,segments_ids
0,0,財務營運風險,財政部充分尊重銀行團確保債權的必要作為，預計元旦後就會請法院實施假扣押，如三重土地順利拍賣，...,"[101, 6512, 3124, 6956, 1041, 1146, 2203, 7028...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,財務營運風險,至三趴以上，「在假扣押前，仍歡迎頂新還錢。」頂新：將持續協商財政部次長吳當傑尊重並同意銀行團...,"[101, 5635, 676, 6640, 809, 677, 8024, 519, 17...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,法律風險,測報告中指出，各款手機在裝上SIM卡前後開機連線時，傳送之資料皆未涉及第一類的敏感資訊，僅為...,"[101, 3947, 1841, 1440, 704, 2900, 1139, 8024,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,法律風險,大哥大，透過4G漫遊方式共網爭議，戰火延燒，威寶電信（台灣之星）及中華電信、遠傳電信等業者，...,"[101, 1920, 1520, 1920, 8024, 6851, 6882, 100,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,法律風險,測報告中指出，各款手機在裝上SIM卡前後開機連線時，傳送之資料皆未涉及第一類的敏感資訊，僅為...,"[101, 3947, 1841, 1440, 704, 2900, 1139, 8024,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
15887,15887,資安風險,"感覺華碩客服有點推託,所謂的個資法是企業有義務妥善保管蒐集來的個人資料不\n感覺華碩客服有點...","[101, 2697, 6221, 5836, 4820, 2145, 3302, 3300...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15888,15888,勞動風險,上市公司玉晶光（3406）今天傳出裁員，對此，公司發言人趙志強表示，並非裁員而\n上市公司玉...,"[101, 677, 2356, 1062, 1385, 4373, 3253, 1045,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15889,15889,勞動風險,繼台企銀（2834）工會因合併問題而罷工，創金融史首例後，大眾銀行（2847）工\n繼台企銀...,"[101, 5262, 1378, 821, 7065, 8020, 11152, 8159...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15890,15890,勞動風險,6億元賣給英業達，獲利21億元，預計明年首季入帳，活化資產。外界認為這將有機會帶動宏達電明年...,"[101, 127, 1023, 1039, 6546, 5183, 5739, 3511,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
